# GIS 495 Final Project Connor Bonham

---

**Overview:**

In this project I will evaluate the correlation between:

* Streamflow vs Water Temperature
* Streamflow vs Elevation
* Streamflow vs Human Wealth

(I originally wanted to use suspended sedement and land use but there are 0 stations in north carolina which measure suspended sedement, and the tif file for land use was too big to push to github)


I will also present spatial data in an interactive StoryMaps using open source products. I will also host a live version of an HTML webpage on GitHub Pages.


---


This code was developed from `storymap.ipynb` of [this Github repository](https://github.com/mdgaines/leaflet_storymap_2023)

That code is adapted from the `index.html` file from [this GitHub repository](https://github.com/HandsOnDataViz/leaflet-map-simple).

That exercise is an adaptation of the ["Leaflet Storymaps with Google Sheets" tutorial](https://handsondataviz.org/leaflet-storymaps-with-google-sheets.html) from the *Hands-On Data Visualization* book by Jack Dougherty & Ilya Ilyankou. The open-acess web edition of the book can be found [here](https://handsondataviz.org/).

Dougherty and Ilyankou developed Leaflet Storymaps with Google Sheets to create an open-source, easily migratable, story map tool. 
I used Mollie D. Gaines's adaptation as a basis for this project.

In [1]:
import pandas as pd
import geopandas as gpd
import folium
import csv
import hydrofunctions as hf
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib as mpl
import matplotlib.patches as mpatches
import numpy as np
import rasterio as rio
import rioxarray as rxr
import os

# Narrative on the Rationale and Objective of the Work
tbd, needs 5 sources

Creating Options CSV

In [2]:
# read in the options csv as a data frame
options_df = pd.read_csv('./csv/Options_Template.csv', index_col=0)

In [3]:
# Personalize Options via indexing
options_df['Customize']['Author Name'] = 'Connor Bonham'
options_df['Customize']['Author Github Repo Link'] = 'https://github.com/ConBonAon/FinalProject2023'
options_df['Customize']['Storymap Title'] = 'Bonham Final Project 2023'
options_df['Customize']['Storymap Subtitle'] = 'GIS 495, Connor Bonham'
options_df['Customize']['Author Email or Website'] = 'connorallenbon@gmail.com'
options_df['Customize']['Narrative Background Color'] = 'lightgray'
# save options as Options.csv
options_df.to_csv('./csv/Options.csv')

In [4]:
options_df.head()

,Customize,Hints
Setting,,
Storymap Info,NaN,"For help, see tutorial in [HandsOnDataViz.org]..."
Storymap Title,Bonham Final Project 2023,NaN
Storymap Subtitle,"GIS 495, Connor Bonham","Add your subtitle, or delete that portion"
Storymap Logo,media/ncsu_logo.jpg,Path to a logo image
Google Analytics Tracking ID,NaN,Sample format: UA-5488840-29


# Reading in Data

## Locations
In North Carolina 37 USGS stations measure for water temperature.

![Temp](./imgs/Water_temp_stations.JPG)

2 are out of commission and 9 are seasonal.

![nonseasonal](./imgs/247Stations.JPG)

Which leaves 26 stations for data gathering.


[1](https://waterdata.usgs.gov/monitoring-location/0351706800/#parameterCode=00065&period=P7D&showMedian=true), [2](https://waterdata.usgs.gov/monitoring-location/03512000/#parameterCode=00065&period=P7D&showMedian=true), [3](https://waterdata.usgs.gov/monitoring-location/03460000/#parameterCode=00065&period=P7D&showMedian=true), [4](https://waterdata.usgs.gov/monitoring-location/03455500/#parameterCode=00065&period=P7D&showMedian=true), [5](https://waterdata.usgs.gov/monitoring-location/03456991/#parameterCode=00065&period=P7D&showMedian=true), [6](https://waterdata.usgs.gov/monitoring-location/03441000/#parameterCode=00065&period=P7D&showMedian=true), [7](https://waterdata.usgs.gov/monitoring-location/03446000/#parameterCode=00065&period=P7D&showMedian=true), [8](https://waterdata.usgs.gov/monitoring-location/03447687/#parameterCode=00065&period=P7D&showMedian=true), [9](https://waterdata.usgs.gov/monitoring-location/03447890/#parameterCode=00010&period=P7D&showMedian=true), [10](https://waterdata.usgs.gov/monitoring-location/03448050/#parameterCode=00010&period=P7D&showMedian=true), [11](https://waterdata.usgs.gov/monitoring-location/0344878100/#parameterCode=00065&period=P7D&showMedian=true), [12](https://waterdata.usgs.gov/monitoring-location/03451200/#parameterCode=00010&period=P7D&showMedian=true), [13](https://waterdata.usgs.gov/monitoring-location/03451500/#parameterCode=00065&period=P7D&showMedian=true), [14](https://waterdata.usgs.gov/monitoring-location/0344894205/#parameterCode=00065&period=P7D&showMedian=true), [15](https://waterdata.usgs.gov/monitoring-location/02137727/#parameterCode=00065&period=P7D&showMedian=true), [16](https://waterdata.usgs.gov/monitoring-location/02111000/#parameterCode=00065&period=P7D&showMedian=true), [17](https://waterdata.usgs.gov/monitoring-location/02101726/#parameterCode=00065&period=P7D&showMedian=true), [18](https://waterdata.usgs.gov/monitoring-location/02077200/#parameterCode=00065&period=P7D&showMedian=true), [19](https://waterdata.usgs.gov/monitoring-location/02077303/#parameterCode=00065&period=P7D&showMedian=true), [20](https://waterdata.usgs.gov/monitoring-location/02087182/#parameterCode=00010&period=P7D&showMedian=true), [21](https://waterdata.usgs.gov/monitoring-location/02102500/#parameterCode=00065&period=P7D&showMedian=true), [22](https://waterdata.usgs.gov/monitoring-location/02105769/#parameterCode=00065&period=P7D&showMedian=true), [23](https://waterdata.usgs.gov/monitoring-location/0208062765/#parameterCode=00065&period=P7D&showMedian=true), [24](https://waterdata.usgs.gov/monitoring-location/02081094/#parameterCode=00065&period=P7D&showMedian=true), [25](https://waterdata.usgs.gov/monitoring-location/0208114150/#parameterCode=00065&period=P7D&showMedian=true), [26](https://waterdata.usgs.gov/monitoring-location/0208458893/#parameterCode=00010&period=P7D&showMedian=true)

### Requesting Data from USGS

I requested 2022 daily data for all 26 locations assuming that they all measured both water temp and streamflow which turned out to be false.

In [5]:
# general objects
site_no = '0351706800,03512000,03460000,03455500,03456991,03441000,03446000,03447687,03447890,03448050,0344878100,03451200,03451500,0344894205,02137727,02111000,02101726,02077200,02077303,02087182,02102500,02105769,0208062765,02081094,0208114150,0208458893' #site no
service = 'dv'
st_date, end_date = '2022-01-01', '2023-01-01'

# flow in ft3/s
daily_mean = hf.NWIS(site=site_no, service=service, start_date=st_date, end_date=end_date)
# What are the properties of the dataset? You can see below that there are two types of data being collected at our site
daily_mean

Requested data from https://waterservices.usgs.gov/nwis/dv/?format=json%2C1.1&sites=0351706800%2C03512000%2C03460000%2C03455500%2C03456991%2C03441000%2C03446000%2C03447687%2C03447890%2C03448050%2C0344878100%2C03451200%2C03451500%2C0344894205%2C02137727%2C02111000%2C02101726%2C02077200%2C02077303%2C02087182%2C02102500%2C02105769%2C0208062765%2C02081094%2C0208114150%2C0208458893&startDT=2022-01-01&endDT=2023-01-01


USGS:02077200: HYCO CREEK NEAR LEASBURG, NC
    00010: <Day>  Temperature, water, degrees Celsius 
    00060: <Day>  Discharge, cubic feet per second 
    00065: <Day>  Gage height, feet 
USGS:02077303: HYCO R BL ABAY D NR MCGEHEES MILL, NC
    00010: <Day>  Temperature, water, degrees Celsius 
    00060: <Day>  Discharge, cubic feet per second 
    00065: <Day>  Gage height, feet 
USGS:0208062765: ROANOKE RIVER AT HALIFAX, NC
    00010: <Day>  Temperature, water, degrees Celsius 
    00065: <Day>  Gage height, feet 
    00095: <Day>  Specific conductance, water, unfiltered, microsiemens per centimeter at 25 degrees Celsius 
    00300: <Day>  Dissolved oxygen, water, unfiltered, milligrams per liter 
    00400: <Day>  pH, water, unfiltered, field, standard units 
USGS:02081094: ROANOKE RIVER AT JAMESVILLE, NC
    00010: <Day>  Temperature, water, degrees Celsius 
    00065: <Day>  Gage height, feet 
    00095: <Day>  Specific conductance, water, unfiltered, microsiemens per centimeter 

In [6]:
daily_mean.df()
daily_mean.df().head()


,USGS:02077200:00010:00001,USGS:02077200:00010:00001_qualifiers,USGS:02077200:00010:00002,USGS:02077200:00010:00002_qualifiers,USGS:02077200:00010:00003,USGS:02077200:00010:00003_qualifiers,USGS:02077200:00060:00003,USGS:02077200:00060:00003_qualifiers,USGS:02077200:00065:00003,USGS:02077200:00065:00003_qualifiers,...,USGS:0351706800:00010:00002,USGS:0351706800:00010:00002_qualifiers,USGS:0351706800:00010:00003,USGS:0351706800:00010:00003_qualifiers,USGS:0351706800:00045:00006,USGS:0351706800:00045:00006_qualifiers,USGS:0351706800:00060:00003,USGS:0351706800:00060:00003_qualifiers,USGS:0351706800:00065:00003,USGS:0351706800:00065:00003_qualifiers
datetimeUTC,,,,,,,,,,,,,,,,,,,,,
2022-01-01 00:00:00+00:00,16.6,A,14.0,A,15.1,A,3.06,A,23.66,A,...,12.4,A,13.3,A,0.21,A,132.0,A,2.97,A
2022-01-02 00:00:00+00:00,16.5,A,15.8,A,16.2,A,5.34,A,23.78,A,...,12.6,A,13.7,A,NaN,hf.missing,329.0,A,3.63,A
2022-01-03 00:00:00+00:00,15.8,A,8.4,A,11.5,A,194.00,A,26.38,A,...,7.4,A,8.9,A,NaN,hf.missing,289.0,A,3.55,A
2022-01-04 00:00:00+00:00,8.4,A,6.0,A,6.8,A,170.00,A,26.31,A,...,6.3,A,6.9,A,NaN,hf.missing,209.0,A,3.30,A
2022-01-05 00:00:00+00:00,7.3,A,5.5,A,6.3,A,52.70,A,24.94,A,...,6.8,A,7.5,A,NaN,hf.missing,185.0,A,3.21,A


In [7]:
streamflow_values = daily_mean.df('q')
streamflow_values

,USGS:02077200:00060:00003,USGS:02077303:00060:00003,USGS:02101726:00060:00003,USGS:02102500:00060:00003,USGS:02105769:00060:00003,USGS:02111000:00060:00003,USGS:02137727:00060:00003,USGS:03441000:00060:00003,USGS:03446000:00060:00003,USGS:03447687:00060:00003,USGS:0344878100:00060:00003,USGS:0344894205:00060:00003,USGS:03451500:00060:00003,USGS:03455500:00060:00003,USGS:03456991:00060:00003,USGS:03460000:00060:00003,USGS:03512000:00060:00003,USGS:0351706800:00060:00003
datetimeUTC,,,,,,,,,,,,,,,,,,
2022-01-01 00:00:00+00:00,3.06,3.57,3.61,853.0,1580.0,16.3,116.0,64.3,83.0,846.0,85.8,12.5,1070.0,44.9,157.0,70.4,398.0,132.0
2022-01-02 00:00:00+00:00,5.34,3.62,9.63,899.0,1990.0,34.6,175.0,135.0,144.0,1330.0,190.0,49.4,1620.0,162.0,410.0,477.0,1760.0,329.0
2022-01-03 00:00:00+00:00,194.00,4.83,1140.00,6830.0,3090.0,93.0,445.0,127.0,168.0,2180.0,329.0,54.6,2970.0,116.0,502.0,340.0,1300.0,289.0
2022-01-04 00:00:00+00:00,170.00,4.86,261.00,18100.0,6440.0,46.2,240.0,98.7,131.0,1740.0,172.0,37.4,2370.0,82.5,351.0,228.0,924.0,209.0
2022-01-05 00:00:00+00:00,52.70,4.31,71.80,18800.0,13100.0,32.2,185.0,87.0,115.0,1350.0,138.0,30.4,1760.0,72.6,298.0,179.0,754.0,185.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-28 00:00:00+00:00,30.60,13.50,18.60,3940.0,5780.0,40.9,181.0,106.0,138.0,1250.0,85.3,20.6,1370.0,73.2,221.0,69.6,305.0,101.0
2022-12-29 00:00:00+00:00,25.90,13.40,16.00,3440.0,5280.0,39.8,177.0,105.0,134.0,1210.0,82.2,20.4,1330.0,75.7,219.0,66.7,296.0,102.0
2022-12-30 00:00:00+00:00,23.40,13.30,13.70,1690.0,4920.0,40.1,178.0,106.0,133.0,1200.0,82.5,21.0,1300.0,78.4,218.0,64.3,292.0,101.0


In [8]:
Watertemp_Values = daily_mean.df('00010')
Watertemp_Values.head()

,USGS:02077200:00010:00001,USGS:02077200:00010:00002,USGS:02077200:00010:00003,USGS:02077303:00010:00001,USGS:02077303:00010:00002,USGS:02077303:00010:00003,USGS:0208062765:00010:00001,USGS:0208062765:00010:00002,USGS:0208062765:00010:00003,USGS:02081094:00010:00001,...,USGS:03456991:00010:00003,USGS:03460000:00010:00001,USGS:03460000:00010:00002,USGS:03460000:00010:00003,USGS:03512000:00010:00001,USGS:03512000:00010:00002,USGS:03512000:00010:00003,USGS:0351706800:00010:00001,USGS:0351706800:00010:00002,USGS:0351706800:00010:00003
datetimeUTC,,,,,,,,,,,,,,,,,,,,,
2022-01-01 00:00:00+00:00,16.6,14.0,15.1,12.4,11.0,11.7,12.9,11.5,12.0,13.4,...,11.6,12.8,10.2,11.6,13.4,11.0,11.9,14.5,12.4,13.3
2022-01-02 00:00:00+00:00,16.5,15.8,16.2,12.9,12.4,12.6,14.1,12.9,13.8,13.7,...,12.6,12.8,10.6,11.6,13.8,11.5,12.5,14.5,12.6,13.7
2022-01-03 00:00:00+00:00,15.8,8.4,11.5,13.3,11.0,11.7,14.2,10.4,12.6,13.9,...,8.9,10.6,5.9,7.8,11.5,6.5,8.9,12.6,7.4,8.9
2022-01-04 00:00:00+00:00,8.4,6.0,6.8,11.0,9.9,10.4,11.6,10.5,11.3,12.9,...,5.9,5.9,4.5,5.3,6.5,5.6,6.0,7.4,6.3,6.9
2022-01-05 00:00:00+00:00,7.3,5.5,6.3,10.0,9.7,9.9,11.1,10.2,10.5,12.4,...,6.1,6.6,5.1,5.7,7.2,5.2,6.1,8.4,6.8,7.5


In [9]:
#making watertemp values the mean version no the min or max
filtered_temp_columns = [col for col in Watertemp_Values.columns if '00003' in col]
Avg_Watertemp_Values = Watertemp_Values[filtered_temp_columns]
Avg_Watertemp_Values

,USGS:02077200:00010:00003,USGS:02077303:00010:00003,USGS:0208062765:00010:00003,USGS:02081094:00010:00003,USGS:0208458893:00010:00003,USGS:02087182:00010:00003,USGS:02101726:00010:00003,USGS:02111000:00010:00003,USGS:02137727:00010:00003,USGS:03446000:00010:00003,USGS:03447687:00010:00003,USGS:03447890:00010:00003,USGS:03448050:00010:00003,USGS:0344878100:00010:00003,USGS:03451200:00010:00003,USGS:03451500:00010:00003,USGS:03456991:00010:00003,USGS:03460000:00010:00003,USGS:03512000:00010:00003,USGS:0351706800:00010:00003
datetimeUTC,,,,,,,,,,,,,,,,,,,,
2022-01-01 00:00:00+00:00,15.1,11.7,12.0,12.8,16.8,13.3,12.5,12.6,12.0,11.9,13.2,13.2,13.2,11.9,12.8,12.8,11.6,11.6,11.9,13.3
2022-01-02 00:00:00+00:00,16.2,12.6,13.8,13.6,17.5,13.1,13.5,14.1,13.6,13.1,14.1,14.0,14.0,13.2,13.9,13.9,12.6,11.6,12.5,13.7
2022-01-03 00:00:00+00:00,11.5,11.7,12.6,13.6,15.8,12.2,12.5,9.8,11.1,9.5,11.5,11.3,11.5,9.6,11.3,11.0,8.9,7.8,8.9,8.9
2022-01-04 00:00:00+00:00,6.8,10.4,11.3,12.5,10.0,11.5,10.7,5.7,7.2,5.3,8.3,8.5,8.6,5.9,8.3,8.1,5.9,5.3,6.0,6.9
2022-01-05 00:00:00+00:00,6.3,9.9,10.5,11.9,10.1,11.4,9.8,5.7,6.7,5.3,7.3,7.4,7.5,5.7,7.3,7.2,6.1,5.7,6.1,7.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-28 00:00:00+00:00,1.4,5.2,6.5,5.5,2.8,7.4,3.2,1.7,2.4,1.2,2.1,1.8,1.9,0.4,1.6,1.7,1.3,-0.0,1.0,3.8
2022-12-29 00:00:00+00:00,2.1,5.5,6.6,5.4,4.2,7.3,3.5,2.6,3.0,2.2,3.0,2.8,2.8,1.9,2.5,2.5,2.4,0.7,2.9,5.1
2022-12-30 00:00:00+00:00,3.6,5.8,6.8,5.6,5.5,7.2,4.0,3.8,4.3,3.4,4.2,3.9,3.9,3.5,3.7,3.7,3.2,2.8,4.1,6.6


20 locations have avg temp, 18 have avg streamflow. 

In [10]:
potential_locations = ["0351706800", "03512000", "03460000", "03455500", "03456991", "03441000", "03446000",
                       "03447687", "03447890", "03448050", "0344878100", "03451200", "03451500", "0344894205",
                       "02137727", "02111000", "02101726", "02077200", "02077303", "02087182", "02102500",
                       "02105769", "0208062765", "02081094", "0208114150", "0208458893"]
# Extract location numbers from both datasets
streamflow_locations = [col.split(":")[1] for col in streamflow_values.columns if 'USGS:' in col and '00060' in col]
watertemp_locations = [col.split(":")[1] for col in Avg_Watertemp_Values.columns if 'USGS:' in col and '00010' in col]

# Find common locations in both datasets
common_locations = set(streamflow_locations).intersection(watertemp_locations)

# Construct column names for the common locations in both datasets
filtered_streamflow_columns = ['USGS:' + loc + ':00060:00003' for loc in common_locations]
filtered_watertemp_columns = ['USGS:' + loc + ':00010:00003' for loc in common_locations]

# Filter columns in both datasets based on the common locations
filtered_streamflow_df = streamflow_values[filtered_streamflow_columns]
filtered_watertemp_df = Avg_Watertemp_Values[filtered_watertemp_columns]



In [11]:
# Display the resulting DataFrames
filtered_streamflow_df

,USGS:03446000:00060:00003,USGS:03451500:00060:00003,USGS:02101726:00060:00003,USGS:03460000:00060:00003,USGS:02111000:00060:00003,USGS:03447687:00060:00003,USGS:02077303:00060:00003,USGS:03456991:00060:00003,USGS:02077200:00060:00003,USGS:0344878100:00060:00003,USGS:03512000:00060:00003,USGS:0351706800:00060:00003,USGS:02137727:00060:00003
datetimeUTC,,,,,,,,,,,,,
2022-01-01 00:00:00+00:00,83.0,1070.0,3.61,70.4,16.3,846.0,3.57,157.0,3.06,85.8,398.0,132.0,116.0
2022-01-02 00:00:00+00:00,144.0,1620.0,9.63,477.0,34.6,1330.0,3.62,410.0,5.34,190.0,1760.0,329.0,175.0
2022-01-03 00:00:00+00:00,168.0,2970.0,1140.00,340.0,93.0,2180.0,4.83,502.0,194.00,329.0,1300.0,289.0,445.0
2022-01-04 00:00:00+00:00,131.0,2370.0,261.00,228.0,46.2,1740.0,4.86,351.0,170.00,172.0,924.0,209.0,240.0
2022-01-05 00:00:00+00:00,115.0,1760.0,71.80,179.0,32.2,1350.0,4.31,298.0,52.70,138.0,754.0,185.0,185.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-28 00:00:00+00:00,138.0,1370.0,18.60,69.6,40.9,1250.0,13.50,221.0,30.60,85.3,305.0,101.0,181.0
2022-12-29 00:00:00+00:00,134.0,1330.0,16.00,66.7,39.8,1210.0,13.40,219.0,25.90,82.2,296.0,102.0,177.0
2022-12-30 00:00:00+00:00,133.0,1300.0,13.70,64.3,40.1,1200.0,13.30,218.0,23.40,82.5,292.0,101.0,178.0


In [12]:
filtered_watertemp_df

,USGS:03446000:00010:00003,USGS:03451500:00010:00003,USGS:02101726:00010:00003,USGS:03460000:00010:00003,USGS:02111000:00010:00003,USGS:03447687:00010:00003,USGS:02077303:00010:00003,USGS:03456991:00010:00003,USGS:02077200:00010:00003,USGS:0344878100:00010:00003,USGS:03512000:00010:00003,USGS:0351706800:00010:00003,USGS:02137727:00010:00003
datetimeUTC,,,,,,,,,,,,,
2022-01-01 00:00:00+00:00,11.9,12.8,12.5,11.6,12.6,13.2,11.7,11.6,15.1,11.9,11.9,13.3,12.0
2022-01-02 00:00:00+00:00,13.1,13.9,13.5,11.6,14.1,14.1,12.6,12.6,16.2,13.2,12.5,13.7,13.6
2022-01-03 00:00:00+00:00,9.5,11.0,12.5,7.8,9.8,11.5,11.7,8.9,11.5,9.6,8.9,8.9,11.1
2022-01-04 00:00:00+00:00,5.3,8.1,10.7,5.3,5.7,8.3,10.4,5.9,6.8,5.9,6.0,6.9,7.2
2022-01-05 00:00:00+00:00,5.3,7.2,9.8,5.7,5.7,7.3,9.9,6.1,6.3,5.7,6.1,7.5,6.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-28 00:00:00+00:00,1.2,1.7,3.2,-0.0,1.7,2.1,5.2,1.3,1.4,0.4,1.0,3.8,2.4
2022-12-29 00:00:00+00:00,2.2,2.5,3.5,0.7,2.6,3.0,5.5,2.4,2.1,1.9,2.9,5.1,3.0
2022-12-30 00:00:00+00:00,3.4,3.7,4.0,2.8,3.8,4.2,5.8,3.2,3.6,3.5,4.1,6.6,4.3


I now have 13 locations where both variables are measured. I want to rename the columns to the same thing so I can compare them by column.

In [13]:
#cutting out USGS: from column names
filtered_streamflow_df.columns = filtered_streamflow_df.columns.str[5:]
filtered_watertemp_df.columns = filtered_watertemp_df.columns.str[5:]




In [14]:
#cutting out :000X0:00003 from column names
filtered_streamflow_df.columns = filtered_streamflow_df.columns.str[:-12]
filtered_watertemp_df.columns = filtered_watertemp_df.columns.str[:-12]

Now that I have the columns names match, I will get a correlation coefficient for streamflow vs water temp for each location. 

In [15]:
# Extract the common column names
common_columns = set(filtered_streamflow_df.columns) & set(filtered_watertemp_df.columns)

# Initialize an empty list to store DataFrames
corrcoef_stmflw_wtrtmp_df = []

# Compute the correlation coefficient for each common column
for column in common_columns:
    data1_column = filtered_streamflow_df[column].values
    data2_column = filtered_watertemp_df[column].values

    correlation_coefficient = np.corrcoef(data1_column, data2_column)[0, 1]

    # Create a temporary DataFrame for each iteration
    temp_df = pd.DataFrame({"Location": [column], "Correlation Coefficient": [correlation_coefficient]})
    
    # Append the temporary DataFrame to the list
    corrcoef_stmflw_wtrtmp_df.append(temp_df)

    print(f"Correlation coefficient for USGS location '{column}' = {correlation_coefficient}")

# Concatenate the list of DataFrames into a single DataFrame
corrcoef_stmflw_wtrtmp_df = pd.concat(corrcoef_stmflw_wtrtmp_df, ignore_index=True)


Correlation coefficient for USGS location '03446000' = -0.006663145379915638
Correlation coefficient for USGS location '03460000' = -0.18623982827422705
Correlation coefficient for USGS location '02101726' = -0.15057422849004087
Correlation coefficient for USGS location '03451500' = -0.07493103512086549
Correlation coefficient for USGS location '02111000' = -0.09153856543663072
Correlation coefficient for USGS location '03447687' = -0.05626917624481335
Correlation coefficient for USGS location '02077303' = -0.13350953287047415
Correlation coefficient for USGS location '03456991' = nan
Correlation coefficient for USGS location '02077200' = nan
Correlation coefficient for USGS location '0344878100' = nan
Correlation coefficient for USGS location '03512000' = nan
Correlation coefficient for USGS location '0351706800' = -0.11225280087809647
Correlation coefficient for USGS location '02137727' = -0.1205493623903371


In [16]:
# Print the resulting DataFrame
print(corrcoef_stmflw_wtrtmp_df)

      Location  Correlation Coefficient
0     03446000                -0.006663
1     03460000                -0.186240
2     02101726                -0.150574
3     03451500                -0.074931
4     02111000                -0.091539
5     03447687                -0.056269
6     02077303                -0.133510
7     03456991                      NaN
8     02077200                      NaN
9   0344878100                      NaN
10    03512000                      NaN
11  0351706800                -0.112253
12    02137727                -0.120549


In [17]:
print(common_locations)

{'03446000', '03451500', '02101726', '03460000', '02111000', '03447687', '02077303', '03456991', '02077200', '0344878100', '03512000', '0351706800', '02137727'}


In [18]:
com_loc = "'03447687', '03446000', '02077200', '03451500', '03456991', '03512000', '0344878100', '02137727', '02111000', '02077303', '0351706800', '03460000', '02101726'"

# Replace single quotes with an empty string
com_loc_mod = com_loc.replace("'", "")

print(com_loc_mod)

03447687, 03446000, 02077200, 03451500, 03456991, 03512000, 0344878100, 02137727, 02111000, 02077303, 0351706800, 03460000, 02101726


Alright so the locations I am going to compare are '03447687', '03446000', '02077200', '03451500', '03456991', '03512000', '0344878100', '02137727', '02111000', '02077303', '0351706800', '03460000', '02101726' and I want to get geospacial data for these locations. 

In [19]:
Com_Loc_df = []

for column in common_columns:
    output = hf.site_file(column)
    site_table = output.table
    latitude = site_table['dec_lat_va'].iloc[0]  
    longitude = site_table['dec_long_va'].iloc[0]
    datum = site_table['dec_coord_datum_cd'].iloc[0]
    alt_land_surf = site_table['alt_va'].iloc[0]
    county = site_table['county_cd'].iloc[0]
    temp_df = pd.DataFrame({"Location": [column], "Lat": [latitude],"Long":[longitude], "Datum":[datum], "County":[county], "Elevation":[alt_land_surf]})
    Com_Loc_df.append(temp_df)
Com_Loc_df = pd.concat(Com_Loc_df, ignore_index=True)
Com_Loc_df

Retrieved the site file for site #03446000 from https://waterservices.usgs.gov/nwis/site/?format=rdb&sites=03446000&siteOutput=expanded&siteStatus=alll
Retrieved the site file for site #03460000 from https://waterservices.usgs.gov/nwis/site/?format=rdb&sites=03460000&siteOutput=expanded&siteStatus=alll
Retrieved the site file for site #02101726 from https://waterservices.usgs.gov/nwis/site/?format=rdb&sites=02101726&siteOutput=expanded&siteStatus=alll
Retrieved the site file for site #03451500 from https://waterservices.usgs.gov/nwis/site/?format=rdb&sites=03451500&siteOutput=expanded&siteStatus=alll
Retrieved the site file for site #02111000 from https://waterservices.usgs.gov/nwis/site/?format=rdb&sites=02111000&siteOutput=expanded&siteStatus=alll
Retrieved the site file for site #03447687 from https://waterservices.usgs.gov/nwis/site/?format=rdb&sites=03447687&siteOutput=expanded&siteStatus=alll
Retrieved the site file for site #02077303 from https://waterservices.usgs.gov/nwis/site

,Location,Lat,Long,Datum,County,Elevation
0,03446000,35.398056,-82.595000,NAD83,89,2088.39
1,03460000,35.667222,-83.072778,NAD83,87,2457.30
2,02101726,35.735141,-79.423075,NAD83,37,482.90
3,03451500,35.608889,-82.578056,NAD83,21,1949.93
4,02111000,35.990833,-81.558333,NAD83,27,1211.30
5,03447687,35.429167,-82.552500,NAD83,89,2032.76
6,02077303,36.522722,-78.997056,NAD83,145,341.97
7,03456991,35.521944,-82.848056,NAD83,87,2581.60
8,02077200,36.397778,-79.196667,NAD83,33,399.30
9,0344878100,35.561389,-82.596944,NAD83,21,1975.32


In [20]:
County_codes = pd.read_csv('./data/County_codes.csv')
County_codes


,County,Name
0,1,Alamance County
1,3,Alexander County
2,5,Alleghany County
3,7,Anson County
4,9,Ashe County
...,...,...
95,191,Wayne County
96,193,Wilkes County
97,195,Wilson County
98,197,Yadkin County


In [21]:

# Merge the datasets on 'county_code'
Com_Loc_County = pd.merge(Com_Loc_df, County_codes, on='County', how='left')

# Display the result
Com_Loc_County

,Location,Lat,Long,Datum,County,Elevation,Name
0,03446000,35.398056,-82.595000,NAD83,89,2088.39,Henderson County
1,03460000,35.667222,-83.072778,NAD83,87,2457.30,Haywood County
2,02101726,35.735141,-79.423075,NAD83,37,482.90,Chatham County
3,03451500,35.608889,-82.578056,NAD83,21,1949.93,Buncombe County
4,02111000,35.990833,-81.558333,NAD83,27,1211.30,Caldwell County
5,03447687,35.429167,-82.552500,NAD83,89,2032.76,Henderson County
6,02077303,36.522722,-78.997056,NAD83,145,341.97,Person County
7,03456991,35.521944,-82.848056,NAD83,87,2581.60,Haywood County
8,02077200,36.397778,-79.196667,NAD83,33,399.30,Caswell County
9,0344878100,35.561389,-82.596944,NAD83,21,1975.32,Buncombe County


## Land Cover

Since the Land cover map is from 2019 I want to grab streamflow data from 2019 to compare it to. I will use the same 13 stations from before for simplicity.

In [22]:
Strmflw_an = []
temp_df = pd.DataFrame(columns=["Location", "Year", "Streamflow"])  # Initialize outside the loop

for column in common_columns:
    output = hf.stats(column, 'annual', missingData='on')
    site_table = output.table
    
    # Filter rows where parameter_cd is '00060' and year_nu is 2019
    filtered_table = site_table.loc[(site_table['parameter_cd'] == '00060') & (site_table['year_nu'] == 2019)]
    
    # Check if there are rows in the filtered_table
    if not filtered_table.empty:
        year = filtered_table['year_nu'].iloc[0]
        stmflw = filtered_table['mean_va'].iloc[0]
        
        # Append the data for the current iteration to temp_df
        temp_df = pd.concat([temp_df, pd.DataFrame({"Location": [column], "Year": [year], "Streamflow": [stmflw]})], ignore_index=True)
    else:
        print(f"No data for {column}")

# final DataFrame
Strmflw_an = temp_df
print(Strmflw_an)


Retrieved annual statistics for site #03446000 from https://waterservices.usgs.gov/nwis/stat//


C:\Users\conno\AppData\Local\Temp\ipykernel_23260\1450280576.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  temp_df = pd.concat([temp_df, pd.DataFrame({"Location": [column], "Year": [year], "Streamflow": [stmflw]})], ignore_index=True)


Retrieved annual statistics for site #03460000 from https://waterservices.usgs.gov/nwis/stat/
Retrieved annual statistics for site #02101726 from https://waterservices.usgs.gov/nwis/stat//
Retrieved annual statistics for site #03451500 from https://waterservices.usgs.gov/nwis/stat//
Retrieved annual statistics for site #02111000 from https://waterservices.usgs.gov/nwis/stat//
Retrieved annual statistics for site #03447687 from https://waterservices.usgs.gov/nwis/stat//
Retrieved annual statistics for site #02077303 from https://waterservices.usgs.gov/nwis/stat//
Retrieved annual statistics for site #03456991 from https://waterservices.usgs.gov/nwis/stat//
Retrieved annual statistics for site #02077200 from https://waterservices.usgs.gov/nwis/stat//
Retrieved annual statistics for site #0344878100 from https://waterservices.usgs.gov/nwis/stat//
Retrieved annual statistics for site #03512000 from https://waterservices.usgs.gov/nwis/stat//
Retrieved annual statistics for site #0351706800 

With this annual streamflow I can look at how each locations streamflow compared to their elevation via a corrcoef.

In [23]:
# Assuming Strmflw_an = df1 is the first dataset and Com_Loc_County = df2 is the second dataset
# with 'Location' as the common column
# 'Streamflow' is the column in df1, and 'Elevation' is the column in df2

# Merge the two datasets on the 'locations' column
Elv_stmfl_df = pd.merge(Strmflw_an, Com_Loc_County, on='Location', how='inner')

# Calculate the correlation coefficient
corrcoeff_Elv_stmfl = np.corrcoef(Elv_stmfl_df['Streamflow'], Elv_stmfl_df['Elevation'])[0, 1]

corrcoeff_Elv_stmfl


0.335436467746548

In [24]:
Elv_stmfl_df

,Location,Year,Streamflow,Lat,Long,Datum,County,Elevation,Name
0,03446000,2019,236.3,35.398056,-82.595000,NAD83,89,2088.39,Henderson County
1,03460000,2019,176.5,35.667222,-83.072778,NAD83,87,2457.30,Haywood County
2,02101726,2019,87.2,35.735141,-79.423075,NAD83,37,482.90,Chatham County
3,03451500,2019,3088.0,35.608889,-82.578056,NAD83,21,1949.93,Buncombe County
4,02111000,2019,76.7,35.990833,-81.558333,NAD83,27,1211.30,Caldwell County
5,03447687,2019,2424.0,35.429167,-82.552500,NAD83,89,2032.76,Henderson County
6,02077303,2019,178.4,36.522722,-78.997056,NAD83,145,341.97,Person County
7,03456991,2019,440.3,35.521944,-82.848056,NAD83,87,2581.60,Haywood County
8,02077200,2019,61.4,36.397778,-79.196667,NAD83,33,399.30,Caswell County
9,0344878100,2019,116.1,35.561389,-82.596944,NAD83,21,1975.32,Buncombe County


When I went to push the code for reading in the land use map, it would not let me push it to git because the tif file was too large. So I abandoned that aspect of the project.

### Chapters

We will add content to our map by creating a `Chapters.csv`.

However, first we will look through our data to get the information we are interested in.

Now, let's read in some of our spatial data.

We can use the `folium` package to see what our Leaflet map will look like.

In [25]:
# Create a map centered at the mean latitude and longitude
map_center = [Com_Loc_df['Lat'].mean(), Com_Loc_df['Long'].mean()]
loc_map = folium.Map(location=map_center, zoom_start=10)

# Add markers for each location
for index, row in Com_Loc_df.iterrows():
    folium.Marker([row['Lat'], row['Long']], popup=row['Location']).add_to(loc_map)

# Save the map as an HTML file or display it in a Jupyter notebook
map_file_name='loc_map.html'

if not os.path.exists(map_file_name):
    # Save the map as an HTML file
    loc_map.save(map_file_name)
    print(f"Map saved as {map_file_name}")
else:
    print(f"A map file named {map_file_name} already exists. Not saving a new map.")
loc_map

A map file named loc_map.html already exists. Not saving a new map.


In [26]:

header = ['Chapter', 'Media Link', 'Media Credit', 'Media Credit Link', 'Description', \
    'Zoom', 'Marker', 'Marker Color', 'Location', 'Latitude', 'Longitude', 'Overlay', 'Overlay Transparency', \
    'GeoJSON Overlay', 'GeoJSON Feature Properties']



In [27]:
# Create a new Chapters.csv

# opens a csv file in write mode
# the with statement makes sure the file closes properly when we're done
with open('./csv/Chapters.csv', 'w', newline='') as options:
    # create the csv writer
    writer = csv.writer(options)

    # write a row to the csv file
    writer.writerow(header)

    writer.writerow(['Title'])

    for column in common_columns:
        writer.writerow([column])

In [28]:
chapter_df = pd.read_csv('./csv/Chapters.csv')
chapter_df

,Chapter,Media Link,Media Credit,Media Credit Link,Description,Zoom,Marker,Marker Color,Location,Latitude,Longitude,Overlay,Overlay Transparency,GeoJSON Overlay,GeoJSON Feature Properties
0,Title,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,03446000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,03460000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,02101726,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,03451500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,02111000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,03447687,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,02077303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,03456991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,02077200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Our first chapter will give a brief overview of Raleigh. We will include a fun skyline image with a link to where we got the image (the Raleigh government webpage). Our map will show a historical, georeferenced map of the Raleigh City Plan from 1834 along with blue markers for each of the 8 schools we will look at for chapters.

To include images in our narative, we need to have them downloaded to our `media` directory.

In [29]:
chapter_df.columns

Index(['Chapter', 'Media Link', 'Media Credit', 'Media Credit Link',
       'Description', 'Zoom', 'Marker', 'Marker Color', 'Location', 'Latitude',
       'Longitude', 'Overlay', 'Overlay Transparency', 'GeoJSON Overlay',
       'GeoJSON Feature Properties'],
      dtype='object')

In [30]:
# path to the image used for this chapter
chapter_df.loc[[0], ['Media Link']] = 'media/cat0.jpg'
# Name of image source
chapter_df.loc[[0], ['Media Credit']] = 'Source: katya-guseva0'
# Link to image
chapter_df.loc[[0], ['Media Credit Link']] = 'https://pixabay.com/photos/cat-sleeping-cat-feline-pet-animal-2605502/'
# Narrative description
chapter_df.loc[[0], ['Description']] = 'This is my final project for GIS 495'
# Zoom level
chapter_df.loc[[0], ['Zoom']] = 15.75
# Markers can be Hidden, Plain, or Numbered
chapter_df.loc[[0], ['Marker']] = 'Hidden'
# if the marker is not Hidden, you can give it a color
chapter_df.loc[[0], ['Marker Color']] = ''
# name of the location
chapter_df.loc[[0], ['Location']] = 'Raleigh, NC'
chapter_df.loc[[0], ['Latitude']] = 35.7796
chapter_df.loc[[0], ['Longitude']] = -78.6382
# If you want to add an overlay of an old map you can search for a georeferenced map at
# mapwraper.net
# add the link to the map in the Overlay section 
chapter_df.loc[[0], ['Overlay']] = 'https://mapwarper.net/maps/tile/51816/{z}/{x}/{y}.png'
# Set how transparent you want the Overlay image/map to be
chapter_df.loc[[0], ['Overlay Transparency']] = 0.9


C:\Users\conno\AppData\Local\Temp\ipykernel_23260\1207726629.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'media/cat0.jpg' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  chapter_df.loc[[0], ['Media Link']] = 'media/cat0.jpg'
C:\Users\conno\AppData\Local\Temp\ipykernel_23260\1207726629.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Source: katya-guseva0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  chapter_df.loc[[0], ['Media Credit']] = 'Source: katya-guseva0'
C:\Users\conno\AppData\Local\Temp\ipykernel_23260\1207726629.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'https://pixabay.com/photos/cat-sleeping-cat-feline-pet-animal-2605502/' has dtype incompatible with float

In [31]:
chapter_df

,Chapter,Media Link,Media Credit,Media Credit Link,Description,Zoom,Marker,Marker Color,Location,Latitude,Longitude,Overlay,Overlay Transparency,GeoJSON Overlay,GeoJSON Feature Properties
0,Title,media/cat0.jpg,Source: katya-guseva0,https://pixabay.com/photos/cat-sleeping-cat-fe...,This is my final project for GIS 495,15.75,Hidden,,"Raleigh, NC",35.7796,-78.6382,https://mapwarper.net/maps/tile/51816/{z}/{x}/...,0.9,NaN,NaN
1,03446000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,03460000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,02101726,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,03451500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,02111000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,03447687,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,02077303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,03456991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,02077200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
Elv_stmfl_df

,Location,Year,Streamflow,Lat,Long,Datum,County,Elevation,Name
0,03446000,2019,236.3,35.398056,-82.595000,NAD83,89,2088.39,Henderson County
1,03460000,2019,176.5,35.667222,-83.072778,NAD83,87,2457.30,Haywood County
2,02101726,2019,87.2,35.735141,-79.423075,NAD83,37,482.90,Chatham County
3,03451500,2019,3088.0,35.608889,-82.578056,NAD83,21,1949.93,Buncombe County
4,02111000,2019,76.7,35.990833,-81.558333,NAD83,27,1211.30,Caldwell County
5,03447687,2019,2424.0,35.429167,-82.552500,NAD83,89,2032.76,Henderson County
6,02077303,2019,178.4,36.522722,-78.997056,NAD83,145,341.97,Person County
7,03456991,2019,440.3,35.521944,-82.848056,NAD83,87,2581.60,Haywood County
8,02077200,2019,61.4,36.397778,-79.196667,NAD83,33,399.30,Caswell County
9,0344878100,2019,116.1,35.561389,-82.596944,NAD83,21,1975.32,Buncombe County


In [33]:
for i in Elv_stmfl_df.index:
    # print(nc_coll_raleigh.iloc[i])
    location = Elv_stmfl_df.loc[[i], ['Location']].values[0][0].title()
    lat = Elv_stmfl_df.loc[[i], ['Lat']].values[0][0]
    lon = Elv_stmfl_df.loc[[i], ['Long']].values[0][0]
    alt = Elv_stmfl_df.loc[[i], ['Elevation']].values[0][0]

    chapter_df.loc[chapter_df['Chapter'] == location, 'Latitude'] = lat
    chapter_df.loc[chapter_df['Chapter'] == location, 'Longitude'] = lon
    chapter_df.loc[chapter_df['Chapter'] == location, 'Elevation'] = alt

    chapter_df.loc[chapter_df['Chapter'] == location, 'Zoom'] = 16
    chapter_df.loc[chapter_df['Chapter'] == location, 'Marker'] = 'Plain'
    chapter_df.loc[chapter_df['Chapter'] == location, 'Marker Color'] = 'blue'

chapter_df


,Chapter,Media Link,Media Credit,Media Credit Link,Description,Zoom,Marker,Marker Color,Location,Latitude,Longitude,Overlay,Overlay Transparency,GeoJSON Overlay,GeoJSON Feature Properties,Elevation
0,Title,media/cat0.jpg,Source: katya-guseva0,https://pixabay.com/photos/cat-sleeping-cat-fe...,This is my final project for GIS 495,15.75,Hidden,,"Raleigh, NC",35.779600,-78.638200,https://mapwarper.net/maps/tile/51816/{z}/{x}/...,0.9,NaN,NaN,NaN
1,03446000,NaN,NaN,NaN,NaN,16.00,Plain,blue,NaN,35.398056,-82.595000,NaN,NaN,NaN,NaN,2088.39
2,03460000,NaN,NaN,NaN,NaN,16.00,Plain,blue,NaN,35.667222,-83.072778,NaN,NaN,NaN,NaN,2457.30
3,02101726,NaN,NaN,NaN,NaN,16.00,Plain,blue,NaN,35.735141,-79.423075,NaN,NaN,NaN,NaN,482.90
4,03451500,NaN,NaN,NaN,NaN,16.00,Plain,blue,NaN,35.608889,-82.578056,NaN,NaN,NaN,NaN,1949.93
5,02111000,NaN,NaN,NaN,NaN,16.00,Plain,blue,NaN,35.990833,-81.558333,NaN,NaN,NaN,NaN,1211.30
6,03447687,NaN,NaN,NaN,NaN,16.00,Plain,blue,NaN,35.429167,-82.552500,NaN,NaN,NaN,NaN,2032.76
7,02077303,NaN,NaN,NaN,NaN,16.00,Plain,blue,NaN,36.522722,-78.997056,NaN,NaN,NaN,NaN,341.97
8,03456991,NaN,NaN,NaN,NaN,16.00,Plain,blue,NaN,35.521944,-82.848056,NaN,NaN,NaN,NaN,2581.60
9,02077200,NaN,NaN,NaN,NaN,16.00,Plain,blue,NaN,36.397778,-79.196667,NaN,NaN,NaN,NaN,399.30


Now, let's save this to a CSV and see how this changes our storymap.

In [34]:
chapter_df.to_csv('./csv/Chapters.csv')

We have a map! We also have some images, links, and narrative text appearing.

Now, let's add a new image and image source information for Peace College.

In [35]:
# Add an image for our first Chapter
chapter_df.loc[1, "Media Link"] = "media/cat1.jpg"

# add source name
chapter_df.loc[1, "Media Credit"] = "Image Source: Katzenspielzeug"

# add the source link
chapter_df.loc[1, "Media Credit Link"] = "https://pixabay.com/photos/cat-pet-animal-tabby-cat-98359/"

In [36]:
# Check our work
chapter_df.head()

,Chapter,Media Link,Media Credit,Media Credit Link,Description,Zoom,Marker,Marker Color,Location,Latitude,Longitude,Overlay,Overlay Transparency,GeoJSON Overlay,GeoJSON Feature Properties,Elevation
0,Title,media/cat0.jpg,Source: katya-guseva0,https://pixabay.com/photos/cat-sleeping-cat-fe...,This is my final project for GIS 495,15.75,Hidden,,"Raleigh, NC",35.779600,-78.638200,https://mapwarper.net/maps/tile/51816/{z}/{x}/...,0.9,NaN,NaN,NaN
1,03446000,media/cat1.jpg,Image Source: Katzenspielzeug,https://pixabay.com/photos/cat-pet-animal-tabb...,NaN,16.00,Plain,blue,NaN,35.398056,-82.595000,NaN,NaN,NaN,NaN,2088.39
2,03460000,NaN,NaN,NaN,NaN,16.00,Plain,blue,NaN,35.667222,-83.072778,NaN,NaN,NaN,NaN,2457.30
3,02101726,NaN,NaN,NaN,NaN,16.00,Plain,blue,NaN,35.735141,-79.423075,NaN,NaN,NaN,NaN,482.90
4,03451500,NaN,NaN,NaN,NaN,16.00,Plain,blue,NaN,35.608889,-82.578056,NaN,NaN,NaN,NaN,1949.93


In [37]:
# Add to the Description of Peace College including when it was founded and an interesting fact about it

chapter_df.loc[1, "Description"] = "This location on the Catawba in Pleasant Gardens, NC had a correlation coefficient of -.12 between water temperature and streamflow. It also has an elevation of 1233.81 and had an average streamflow of 368.8 in 2019. It is located in Mcdowell County"

chapter_df.head()

,Chapter,Media Link,Media Credit,Media Credit Link,Description,Zoom,Marker,Marker Color,Location,Latitude,Longitude,Overlay,Overlay Transparency,GeoJSON Overlay,GeoJSON Feature Properties,Elevation
0,Title,media/cat0.jpg,Source: katya-guseva0,https://pixabay.com/photos/cat-sleeping-cat-fe...,This is my final project for GIS 495,15.75,Hidden,,"Raleigh, NC",35.779600,-78.638200,https://mapwarper.net/maps/tile/51816/{z}/{x}/...,0.9,NaN,NaN,NaN
1,03446000,media/cat1.jpg,Image Source: Katzenspielzeug,https://pixabay.com/photos/cat-pet-animal-tabb...,This location on the Catawba in Pleasant Garde...,16.00,Plain,blue,NaN,35.398056,-82.595000,NaN,NaN,NaN,NaN,2088.39
2,03460000,NaN,NaN,NaN,NaN,16.00,Plain,blue,NaN,35.667222,-83.072778,NaN,NaN,NaN,NaN,2457.30
3,02101726,NaN,NaN,NaN,NaN,16.00,Plain,blue,NaN,35.735141,-79.423075,NaN,NaN,NaN,NaN,482.90
4,03451500,NaN,NaN,NaN,NaN,16.00,Plain,blue,NaN,35.608889,-82.578056,NaN,NaN,NaN,NaN,1949.93


In [38]:
Elv_stmfl_df

,Location,Year,Streamflow,Lat,Long,Datum,County,Elevation,Name
0,03446000,2019,236.3,35.398056,-82.595000,NAD83,89,2088.39,Henderson County
1,03460000,2019,176.5,35.667222,-83.072778,NAD83,87,2457.30,Haywood County
2,02101726,2019,87.2,35.735141,-79.423075,NAD83,37,482.90,Chatham County
3,03451500,2019,3088.0,35.608889,-82.578056,NAD83,21,1949.93,Buncombe County
4,02111000,2019,76.7,35.990833,-81.558333,NAD83,27,1211.30,Caldwell County
5,03447687,2019,2424.0,35.429167,-82.552500,NAD83,89,2032.76,Henderson County
6,02077303,2019,178.4,36.522722,-78.997056,NAD83,145,341.97,Person County
7,03456991,2019,440.3,35.521944,-82.848056,NAD83,87,2581.60,Haywood County
8,02077200,2019,61.4,36.397778,-79.196667,NAD83,33,399.30,Caswell County
9,0344878100,2019,116.1,35.561389,-82.596944,NAD83,21,1975.32,Buncombe County


In [39]:
print(corrcoef_stmflw_wtrtmp_df)

      Location  Correlation Coefficient
0     03446000                -0.006663
1     03460000                -0.186240
2     02101726                -0.150574
3     03451500                -0.074931
4     02111000                -0.091539
5     03447687                -0.056269
6     02077303                -0.133510
7     03456991                      NaN
8     02077200                      NaN
9   0344878100                      NaN
10    03512000                      NaN
11  0351706800                -0.112253
12    02137727                -0.120549


In [40]:
census = gpd.read_file('./csv/censusdata 2020/ACSST5Y2020.S1901-Data.csv')

In [41]:
census.crs = "EPSG:4269"

In [42]:
columns_to_keep = ['GEO_ID', 'NAME', 'S1901_C01_001E', 'S1901_C01_013E']
t_census = census[columns_to_keep]

In [43]:
t_census

,GEO_ID,NAME,S1901_C01_001E,S1901_C01_013E
0,Geography,Geographic Area Name,Estimate!!Households!!Total,Estimate!!Households!!Mean income (dollars)
1,0400000US37,North Carolina,4031592,79620
2,1400000US37001020100,"Census Tract 201, Alamance County, North Carolina",1842,71869
3,1400000US37001020200,"Census Tract 202, Alamance County, North Carolina",1349,37764
4,1400000US37001020301,"Census Tract 203.01, Alamance County, North Ca...",1422,46863
...,...,...,...,...
2669,1400000US37199960101,"Census Tract 9601.01, Yancey County, North Car...",1476,58838
2670,1400000US37199960102,"Census Tract 9601.02, Yancey County, North Car...",826,48273
2671,1400000US37199960200,"Census Tract 9602, Yancey County, North Carolina",1887,66153
2672,1400000US37199960300,"Census Tract 9603, Yancey County, North Carolina",1299,53214


In [44]:
print(census.crs)

EPSG:4269


### Colormaps and GeoJSONs

In [ ]:
from matplotlib import colormaps
# list(colormaps)
# from colorspacious import cspace_converter

import matplotlib.pyplot as plt
import numpy as np

import matplotlib as mpl

from pylab import cm

In [ ]:
# Set up to visualize the colors you want to use:

# number of classes you need
n_classes = 256
gradient = np.linspace(0, 1, n_classes)
gradient = np.vstack((gradient, gradient))

In [ ]:
# Choose the color ramp you want
colormap_name = 'BuPu'

# Plot the color map
plt.imshow(gradient, aspect='auto', cmap=mpl.colormaps[colormap_name])

In [ ]:
# number of classes you need
n_classes = 10
gradient = np.linspace(0, 1, n_classes)
gradient = np.vstack((gradient, gradient))

# Choose the color ramp you want
colormap_name = 'BuPu'

# Plot the color map
plt.imshow(gradient, aspect='auto', cmap=mpl.colormaps[colormap_name])

#### Exploring colormaps
Check out more details on colormaps [here](https://matplotlib.org/stable/users/explain/colors/colormaps.html)

In [ ]:
# adapted from matplotlib documentation
def plot_color_gradients(category, cmap_list, gradient):
    # Create figure and adjust figure height to number of colormaps
    nrows = len(cmap_list)
    figh = 0.35 + 0.15 + (nrows + (nrows - 1) * 0.1) * 0.22
    fig, axs = plt.subplots(nrows=nrows + 1, figsize=(8, figh+2.5))
    fig.subplots_adjust(top=1 - 0.35 / figh, bottom=0.25 / figh,
                        left=0.2, right=0.99)
    axs[0].set_title(f'{category} colormaps', fontsize=16)

    for ax, name in zip(axs, cmap_list):
        ax.imshow(gradient, aspect='auto', cmap=mpl.colormaps[name])
        ax.text(-0.01, 0.5, name, va='center', ha='right', fontsize=14,
                transform=ax.transAxes)

    # Turn off *all* ticks & spines, not just the ones with colormaps.
    for ax in axs:
        ax.set_axis_off()

    # # Save colormap list for later.
    # cmaps[category] = cmap_list

In [ ]:
plot_color_gradients('Perceptually Uniform Sequential',
                     ['viridis', 'plasma', 'inferno', 'magma', 'cividis'],
                     gradient)

In [ ]:
plot_color_gradients('Sequential',
                     ['Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
                      'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
                      'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn'],
                      gradient)

In [ ]:
plot_color_gradients('Diverging',
                     ['PiYG', 'PRGn', 'BrBG', 'PuOr', 'RdGy', 'RdBu', 'RdYlBu',
                      'RdYlGn', 'Spectral', 'coolwarm', 'bwr', 'seismic'],
                      gradient)

#### Get colormap values to use in GeoJSON

In [ ]:
def plot_hex_color_gradients(cmap, rows:int=2, columns:int=5):
    
    # Get list of hexidecimal color values
    cmap_hex_lst = [mpl.colors.rgb2hex(cmap(c)) for c in range(cmap.N)]
    
    # Reorder hex labels in a snake pattern - start with top left, end bottom left
    cmap_hex_lst_reordered = np.reshape(cmap_hex_lst, (rows, columns))
    cmap_hex_lst_reordered[1] = np.flip(cmap_hex_lst_reordered[1], 0)

    # Reorder color values in a snake pattern - start with top left, end bottom left
    cmap_col_reordered = np.reshape(list(range(cmap.N)), (rows, columns))
    cmap_col_reordered[1] = np.flip(cmap_col_reordered[1], 0)


    # Plot
    fig, ax = plt.subplots(figsize=(cmap.N, columns / rows + 1))
    im = ax.imshow(cmap_col_reordered, aspect='auto', cmap=cmap)

    # Loop over data dimensions and create text annotations.
    for i in range(rows):
        if i == 0:
            text_col = 'black'
        else:
            text_col = 'w'
        for j in range(columns):
            text = ax.text(j, i, cmap_hex_lst_reordered[i, j],
                        ha="center", va="center", color=text_col,
                        size=16)
    # Remove axis titles    
    ax.set_axis_off()
    
    # Add name of colormap
    ax.set_title(cmap.name, size = 20)

    # return regular ordered hex color values
    return(cmap_hex_lst)

In [ ]:
cmap = cm.get_cmap('BuPu', 10)
cmap


In [ ]:
cmap_hex_values = plot_hex_color_gradients(cmap)

In [ ]:
cmap_hex_values

In [ ]:
print(
f'''
min: {np.min(wake_co['AWATER'])}
10: {np.percentile(wake_co['AWATER'], 10)}
20: {np.percentile(wake_co['AWATER'], 20)}
30: {np.percentile(wake_co['AWATER'], 30)}
40: {np.percentile(wake_co['AWATER'], 40)}
50: {np.percentile(wake_co['AWATER'], 50)}
60: {np.percentile(wake_co['AWATER'], 60)}
70: {np.percentile(wake_co['AWATER'], 70)}
80: {np.percentile(wake_co['AWATER'], 80)}
90: {np.percentile(wake_co['AWATER'], 90)}
max: {np.max(wake_co['AWATER'])}
''')

In [ ]:
wake_co['fillColor'] = cmap_hex_values[0]

for i in range(cmap.N):
    # print((i+1) * 10)
    print(np.percentile(wake_co['AWATER'], (i+1) * 10))
    print(cmap_hex_values[i])
    wake_co.loc[wake_co['AWATER'] >= np.percentile(wake_co['AWATER'], (i+1) * 10), 'fillColor']  = cmap_hex_values[i]
wake_co


In [ ]:
wake_co.plot(color=wake_co['fillColor'])


In [ ]:
wake_co.to_file('./geojson/wake_co_census_colored_AWATER.geojson', driver='GeoJSON')


In [ ]:
chapter_df.loc[chapter_df['Chapter']=='Wake Technical Community College', 'GeoJSON Overlay'] = 'geojson/wake_co_census_colored_AWATER.geojson'
chapter_df.loc[chapter_df['Chapter']=='Wake Technical Community College', 'GeoJSON Feature Properties'] = 'fillColor:gray;color:white'


In [ ]:
chapter_df.to_csv('./csv/Chapters.csv')